In [4]:
import numpy as np
import random

# 1. 상태 공간 정의
states = ["Sunny", "Cloudy", "Rainy"]

# 2. 전이 행렬 (Transition Matrix) 정의
# 행(Row): 현재 상태 / 열(Column): 다음 상태
# 순서: [Sunny, Cloudy, Rainy]
# 예: transition_matrix[0][1]은 Sunny -> Cloudy 확률 (0.3)
transition_matrix = [
    [0.6, 0.3, 0.1],  # Sunny일 때
    [0.3, 0.4, 0.3],  # Cloudy일 때
    [0.2, 0.3, 0.5]   # Rainy일 때
]

# 확률 행렬 검증 (각 행의 합은 1이어야 함)
for i, row in enumerate(transition_matrix):
    assert sum(row) == 1.0, f"Row {i} probabilities must sum to 1"

def simulate_weather(days, start_state="Sunny"):
    """
    주어진 일수(days) 동안 날씨를 시뮬레이션하는 함수
    """
    current_state = start_state
    weather_sequence = [current_state]

    print(f"--- 시뮬레이션 시작 (초기 상태: {start_state}) ---")

    for i in range(days):
        # 현재 상태의 인덱스 찾기
        curr_idx = states.index(current_state)

        # 현재 상태에 따른 다음 상태의 확률 분포 가져오기
        probs = transition_matrix[curr_idx]

        # 확률에 따라 다음 상태 결정 (np.random.choice 사용)
        next_state = np.random.choice(states, p=probs)

        weather_sequence.append(next_state)
        current_state = next_state

    return weather_sequence

# --- 실행 ---
# 7일간의 날씨 변화 시뮬레이션
simulation_days = 7
result = simulate_weather(simulation_days, start_state="Sunny")

print("\n[결과: 7일간의 날씨 변화]")
print(" -> ".join(result))

# 상태별 발생 횟수 통계
from collections import Counter
counts = Counter(result)
print(f"\n[통계] 맑음: {counts['Sunny']}회, 흐림: {counts['Cloudy']}회, 비: {counts['Rainy']}회")

--- 시뮬레이션 시작 (초기 상태: Sunny) ---

[결과: 7일간의 날씨 변화]
Sunny -> Cloudy -> Cloudy -> Cloudy -> Cloudy -> Cloudy -> Cloudy -> Sunny

[통계] 맑음: 2회, 흐림: 6회, 비: 0회


In [1]:
import random

class MarkovTextGenerator:
    def __init__(self):
        # 단어의 연결 관계를 저장할 딕셔너리
        # 예: {'나는': ['학교에', '집에', '밥을'], '학교에': ['갑니다', '있습니다']}
        self.chain = {}

    def train(self, text):
        """
        텍스트 데이터를 읽어 단어 간의 전이 확률(연결 관계)을 학습합니다.
        """
        # 1. 텍스트를 공백 기준으로 단어 분리
        words = text.split()

        # 2. 현재 단어와 다음 단어를 짝지어 딕셔너리에 저장
        for i in range(len(words) - 1):
            current_word = words[i]
            next_word = words[i + 1]

            if current_word not in self.chain:
                self.chain[current_word] = []

            self.chain[current_word].append(next_word)

        print(f"학습 완료! 총 {len(self.chain)}개의 단어 관계를 학습했습니다.")

    def generate(self, start_word, length=10):
        """
        시작 단어 주어지면 마르코프 체인을 따라 문장을 생성합니다.
        """
        if start_word not in self.chain:
            return "오류: 학습된 데이터에 없는 시작 단어입니다."

        # 시작 단어로 문장 시작
        current_word = start_word
        sentence = [current_word]

        for _ in range(length - 1):
            # 현재 단어 뒤에 올 수 있는 단어 목록을 가져옴
            next_words = self.chain.get(current_word)

            # 더 이상 이어질 단어가 없으면 중단
            if not next_words:
                break

            # 목록 중 하나를 랜덤하게 선택 (확률적 선택)
            next_word = random.choice(next_words)

            sentence.append(next_word)
            current_word = next_word

        return " ".join(sentence)

# --- 실행 예시 ---

# 1. 학습할 텍스트 데이터 (반복되는 패턴이 있을수록 재미있는 결과가 나옵니다)
training_data = """
기사는 검을 들었다. 기사는 용을 향해 달렸다.
용은 불을 뿜었다. 불을 피한 기사는 용을 공격했다.
용은 하늘로 날아올랐다. 기사는 하늘을 쳐다보았다.
공주는 기사를 걱정했다. 기사는 공주를 위해 싸웠다.
검을 든 기사는 용감했다. 용은 무서운 소리를 냈다.
"""

# 2. 봇 생성 및 학습
bot = MarkovTextGenerator()
bot.train(training_data)

# 3. 문장 생성 테스트
print("\n[생성된 문장들]")
print("1.", bot.generate("기사는", length=5))
print("2.", bot.generate("용은", length=5))
print("3.", bot.generate("불을", length=6))

학습 완료! 총 25개의 단어 관계를 학습했습니다.

[생성된 문장들]
1. 기사는 용감했다. 용은 불을 뿜었다.
2. 용은 불을 뿜었다. 불을 피한
3. 불을 피한 기사는 검을 들었다. 기사는


In [3]:
!pip install markovify
import requests
import markovify

def load_gutenberg_text(url):
    """
    URL에서 텍스트를 다운로드하고, 구텐베르크의 헤더/푸터(라이선스 고지 등)를 제거합니다.
    """
    print(f"다운로드 중... {url}")
    response = requests.get(url)
    response.encoding = 'utf-8' # 인코딩 설정
    text = response.text

    # 구텐베르크 텍스트에는 소설 내용 앞뒤로 긴 라이선스 설명이 붙어 있습니다.
    # 이를 제거하고 순수 본문만 추출하는 과정입니다.

    # 본문 시작과 끝을 알리는 마커 (책마다 조금씩 다를 수 있으나 보통 이렇습니다)
    start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK"
    end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK"

    start_idx = text.find(start_marker)
    end_idx = text.find(end_marker)

    # 마커를 찾았다면 그 사이의 텍스트만, 못 찾았다면 전체 텍스트 반환
    if start_idx != -1 and end_idx != -1:
        # start_marker 줄바꿈 이후부터 end_marker 전까지
        real_text = text[start_idx:end_idx]
        # 앞부분의 'START...' 문구 제거를 위해 조금 더 다듬기
        real_text = real_text.split('\n', 1)[-1]
        return real_text

    return text

# --- 메인 실행부 ---

# 1. 데이터 가져오기 (이상한 나라의 앨리스 - Lewis Carroll)
url = "https://www.gutenberg.org/files/11/11-0.txt"
text_data = load_gutenberg_text(url)

print(f"\n[데이터 로드 완료] 텍스트 길이: {len(text_data)}자")

# 2. Markovify 모델 빌드 (State Size=2)
# state_size=2는 '현재 단어' 1개가 아니라 '앞의 두 단어'를 보고 다음을 예측한다는 뜻입니다.
# 문맥이 훨씬 자연스러워집니다.
text_model = markovify.Text(text_data, state_size=2)

print("모델 학습 완료! 문장을 생성합니다...\n")
print("-" * 50)

# 3. 문장 생성 (5개 시도)
for i in range(5):
    # make_sentence: 문장의 시작과 끝(마침표 등)을 고려해 자연스러운 문장을 만듭니다.
    sentence = text_model.make_sentence(tries=100)

    if sentence:
        print(f"{i+1}. {sentence}")
    else:
        print(f"{i+1}. (문장 생성 실패 - 조건에 맞는 문장을 못 찾음)")

print("-" * 50)

# 4. 짧은 문장 생성 (옵션)
print("\n[짧은 문장 생성 예시 (140자 이내)]")
print(text_model.make_short_sentence(140))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 7.2 MB/s eta 0:00:00
다운로드 중... https://www.gutenberg.org/files/11/11-0.txt

[데이터 로드 완료] 텍스트 길이: 144602자
모델 학습 완료! 문장을 생성합니다...

--------------------------------------------------
1. But there seemed to be nothing but the Rabbit came up to her that she might as well wait, as she listened, or seemed to listen, the whole pack rose up into hers—she could hear the very tones of her voice, and see what was going to give the hedgehog a blow with its mouth open, gazing up into the sky.
2. The poor little thing sat down and looked along the passage into the garden with one elbow against the roof of the hall: in fact she was now the right size, that it was sneezing and howling alternately without a moment’s pause.
3. Luckily for Alice, the little glass table.
4. And here poor Alice began telling them her adventures from the trees as well wait, as she could for sneezing.
5. Let me think: was I the same age as herself, to see that queer li